In [54]:
%load_ext autoreload
%autoreload 2

import fibsem
from fibsem import utils, acquire, movement, calibration
from fibsem.structures import BeamType
import numpy as np

from pprint import pprint
import os


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
microscope, settings = utils.setup_session()

2022-09-07 16:29:15,834 — root — INFO — connect_to_microscope:28 — Microscope client connecting to [10.0.0.1]
2022-09-07 16:29:15,839 — root — INFO — connect_to_microscope:31 — Microscope client connected to [10.0.0.1]
2022-09-07 16:29:15,840 — root — INFO — setup_session:242 — Finished setup for session: demo_2022-09-07.04-29-15PM


Client connecting to [10.0.0.1:7520]...
Client connected to [10.0.0.1:7520]


In [31]:
def take_image_stack(microscope, settings, hfws, label):
    
    for hfw in hfws:
        settings.image.hfw = hfw
        settings.image.label = f"{label}_{hfw:.5f}"
    
        ib_image = acquire.new_image(microscope, settings.image)


def post_light_image_workflow(microscope, settings, state, hfws, diag_shift):
    
    diag_dx = np.sqrt(2) / 2  * diag_shift
    diag_dy = np.sqrt(2) / 2  * diag_shift
    
    # no move
    calibration.set_microscope_state(microscope, state)
    take_image_stack(microscope, settings, hfws, "base")

    # move diag down
    calibration.set_microscope_state(microscope, state)
    movement.move_stage_relative_with_corrected_movement(
        microscope, 
        settings,
        dx=-diag_dx,
        dy=-diag_dy,
        beam_type=BeamType.ION     
    )
    take_image_stack(microscope, settings, hfws, "diag_down")

    # move diag up 
    calibration.set_microscope_state(microscope, state)
    movement.move_stage_relative_with_corrected_movement(
        microscope, 
        settings,
        dx=diag_dx,
        dy=diag_dy,
        beam_type=BeamType.ION     
    )
    take_image_stack(microscope, settings, hfws, "diag_up")



In [52]:
settings.image.beam_type = BeamType.ION
settings.image.dwell_time = 200e-9
settings.image.save = True
settings.image.resolution = "1536x1024"
settings.image.save_path = "correlation"
hfws = [50e-6, 80e-6, 150e-6, 200e-6, 300e-6, 500e-6]

tile_shift = 40e-6
diag_shift = 40e-6


2.8284271247461906e-05 2.8284271247461906e-05 4e-05


In [33]:
# initial position
initial_position = calibration.get_current_microscope_state(microscope)

In [53]:
N_GRID_Y = 3
N_GRID_X = 3

for grid_y in range(N_GRID_Y):
    for grid_x in range(N_GRID_X):

        # set folder for grid step
        settings.image.save_path = os.path.join("correlation", f"{grid_y:03}.{grid_x:03}")

        # take base images and state
        base_state = calibration.get_current_microscope_state(microscope)
        take_image_stack(microscope, settings, hfws, "start")

        # move to piescope, take light image

        # paause for piescope workflow
        input("PIESCOPE WORKFLOW")

        # post piescope workflow
        post_light_image_workflow(microscope, settings, base_state, hfws, diag_shift)

        # move stage to next tile column
        calibration.set_microscope_state(microscope, base_state)
        acquire.new_image(microscope, settings.image)
        movement.move_stage_relative_with_corrected_movement(
            microscope, settings, dx=tile_shift, dy=0, beam_type=BeamType.ION 
        )

    # move back to initial grid position
    calibration.set_microscope_state(initial_position)

    # move to next grid row
    movement.move_stage_relative_with_corrected_movement(
        microscope, settings, dx=0, dy=tile_shift, beam_type=BeamType.ION 
    )
    initial_position = calibration.get_current_microscope_state(microscope)

2022-09-07 16:50:52,075 — root — INFO — set_microscope_state:234 — restoring microscope state...
2022-09-07 16:50:52,734 — root — INFO — safe_absolute_stage_movement:338 — safe moving to StagePosition(x=-0.00428725, y=-0.002529, z=0.031460134, t=0.43633351, r=-2.2863664, coordinate_system=Raw)
2022-09-07 16:50:53,097 — root — INFO — safe_absolute_stage_movement:340 — safe movement complete.
2022-09-07 16:50:53,098 — root — INFO — set_microscope_state:244 — restoring electron beam settings...
2022-09-07 16:51:08,346 — root — INFO — set_microscope_state:265 — restoring ion beam settings...
2022-09-07 16:51:08,669 — root — INFO — set_microscope_state:283 — microscope state restored
2022-09-07 16:51:08,754 — root — INFO — autocontrast:27 — automatically adjusting contrast...
2022-09-07 16:51:12,762 — root — INFO — acquire_image:187 — acquiring new ION image.
2022-09-07 16:51:15,043 — root — INFO — gamma_correction:86 — GAMMA_CORRECTION | Ion | 17.799 | 1.000
2022-09-07 16:51:15,148 — root 

AdornedImage(width=1536, height=1024, bit_depth=8)

In [82]:


arr = np.zeros((500, 500))

cx, cy = arr.shape[1] // 2, arr.shape[0] // 2
print(cx, cy)

crop = arr[:cy, :]
print(crop.shape)

250 250
(250, 500)


1536

In [87]:
grid_y = 0
grid_x = 1
settings.image.save_path = os.path.join("correlation", f"{grid_y:03}.{grid_x:03}")

In [88]:
print(settings.image.save_path)

correlation\000.001
